In [1]:
from matplotlib import pyplot as plt
# multi-step encoder-decoder lstm 
import tensorflow as tf
from numpy import array
from tensorflow.keras import layers as L
from tensorflow.keras import models as M

import time
import datetime    #second to hour

import numpy as np
import pandas as pd


#should be sorted for index's conformity
LABELS = [0, 1, 2, 3, 4, 5] #"clap", "kick_ball", "kiss", "punch", "shoot_gun", dummy

size_grid = 640
size_sample = 15  #input size + predict output size(1)  (timesteps)


#sampling start
startTime = time.time()



#Load the pre dataset

df= pd.DataFrame(columns = ['label', 'video', 'segment', 'frame', 'change'])

f = open('createdData50_50_fin.txt', mode = 'rt')

#Get all data to dataframe
while True:
    line = f.readline()
    if not line:
        break
    
    parts = line.split(' ')
    
    framechange = []
    for change in range(size_grid-1):
        framechange.append(int(parts[5 + change]))
    framechange.append(int(parts[4+size_grid].split('\n')[0]))
    
    df = df.append({'label' : parts[0], 'video' : parts[1], 'segment' : parts[2], 'frame' : parts[3], 'change' : framechange },
                              ignore_index = True)
    
#Sort by Ascending
df.sort_values(by=['label', 'video', 'segment', 'frame'])

print(df)

print("-------------------Dataset load finished----------------------")
print("--------------------  Sampling Start  ------------------------")


label = ''
video_index = 0

segments = []
segment_index = -1

count_segments = 0

count_video_samples = 0
count_total_samples = []

total_samples = []
total_data = [] #label included


#####리팩토링시 삭제할 부분 ... (데이터셋 확충 이후)
#check = 0
#total_oneclass = 0
#temp = 0

for i in df.index:
    
    if label != df['label'][i] :
        if label != '':          #이전 레이블에서 넘어갔을때

            ###### 데이터셋 확충 이후 삭제
            #check = 1
            #temp = count_video_samples
            #total_oneclass = temp
            
            
            #초기화    
            segment_index = 0
            segments = []
            count_segments = 0
            count_total_samples.append(count_video_samples)
            count_video_samples = 0
            total_data.append([label, total_samples])    
            print('--Action : ' + label + ' finished--')
            total_samples = []

            
            
        label = df['label'][i]
        video_index = df['video'][i]
        segment_index = df['segment'][i]
        
        
    if video_index != df['video'][i]:
        segment_index = df['segment'][i]
        segments = []
        count_segments = 0
        print('--%dth video sampled, samples : %d / total samples : %d--'%(int(video_index), count_video_samples, len(total_samples)))
        count_video_samples = 0
        video_index = df['video'][i]
                        
    if segment_index != df['segment'][i]:
        segment_index = df['segment'][i]
        segments = []
        count_segments = 0
    
    count_segments += 1
    segments.append(df['change'][i])
    
        
    #Sample 갯수에 도달하면 sample에 넣고 segment 초기화    
    if count_segments == size_sample :
        ############################ to make the dataset size same (나중에 고칠때 데이터셋은 확충해서 이 부분 없애자)
        #if check == 1:
            #if count_video_samples >= total_oneclass :
               # continue
            
            
        total_samples.append(segments)
        count_segments = 0
        segments = []
        count_video_samples += 1
              
count_total_samples.append(count_video_samples)            
total_data.append([label, total_samples])


print('--Action : ' + label + ' finished--')

x_dummy = 510*np.random.random((int(len(total_samples)*2/3), size_sample, size_grid))
label = LABELS[-1]
total_data.append([label, x_dummy])

print('--Dummy Class Samples : %d Added--'%int(len(total_samples)*2/3))

print("--------------------  Sampling Finished  ------------------------")


endTime = time.time() - startTime
hourtime = str(datetime.timedelta(seconds = endTime))
print("====== " + hourtime + " ======")

           label video segment frame  \
0           clap     1       1     4   
1           clap     1       1     5   
2           clap     1       1     6   
3           clap     1       1     7   
4           clap     1       3    14   
...          ...   ...     ...   ...   
48264  shoot_gun   103       1   105   
48265  shoot_gun   103       1   106   
48266  shoot_gun   103       1   107   
48267  shoot_gun   103       1   108   
48268  shoot_gun   103       1   109   

                                                  change  
0      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
...                                                  ...  
48264  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, ...  
48265  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

--127th video sampled, samples : 0 / total samples : 218--
--128th video sampled, samples : 0 / total samples : 218--
--129th video sampled, samples : 15 / total samples : 233--
--Action : clap finished--
--1th video sampled, samples : 1 / total samples : 1--
--2th video sampled, samples : 0 / total samples : 1--
--3th video sampled, samples : 0 / total samples : 1--
--4th video sampled, samples : 0 / total samples : 1--
--5th video sampled, samples : 0 / total samples : 1--
--6th video sampled, samples : 0 / total samples : 1--
--7th video sampled, samples : 3 / total samples : 4--
--8th video sampled, samples : 0 / total samples : 4--
--9th video sampled, samples : 3 / total samples : 7--
--10th video sampled, samples : 2 / total samples : 9--
--11th video sampled, samples : 0 / total samples : 9--
--12th video sampled, samples : 0 / total samples : 9--
--13th video sampled, samples : 0 / total samples : 9--
--14th video sampled, samples : 0 / total samples : 9--
--15th video sampled

--29th video sampled, samples : 2 / total samples : 143--
--30th video sampled, samples : 1 / total samples : 144--
--31th video sampled, samples : 1 / total samples : 145--
--32th video sampled, samples : 1 / total samples : 146--
--33th video sampled, samples : 0 / total samples : 146--
--34th video sampled, samples : 0 / total samples : 146--
--35th video sampled, samples : 5 / total samples : 151--
--36th video sampled, samples : 8 / total samples : 159--
--37th video sampled, samples : 3 / total samples : 162--
--38th video sampled, samples : 2 / total samples : 164--
--39th video sampled, samples : 1 / total samples : 165--
--40th video sampled, samples : 1 / total samples : 166--
--41th video sampled, samples : 11 / total samples : 177--
--42th video sampled, samples : 3 / total samples : 180--
--43th video sampled, samples : 1 / total samples : 181--
--44th video sampled, samples : 2 / total samples : 183--
--45th video sampled, samples : 4 / total samples : 187--
--46th video 

--83th video sampled, samples : 0 / total samples : 84--
--84th video sampled, samples : 7 / total samples : 91--
--85th video sampled, samples : 0 / total samples : 91--
--86th video sampled, samples : 0 / total samples : 91--
--87th video sampled, samples : 0 / total samples : 91--
--88th video sampled, samples : 5 / total samples : 96--
--89th video sampled, samples : 4 / total samples : 100--
--90th video sampled, samples : 0 / total samples : 100--
--91th video sampled, samples : 0 / total samples : 100--
--92th video sampled, samples : 0 / total samples : 100--
--93th video sampled, samples : 0 / total samples : 100--
--94th video sampled, samples : 0 / total samples : 100--
--95th video sampled, samples : 0 / total samples : 100--
--96th video sampled, samples : 0 / total samples : 100--
--97th video sampled, samples : 2 / total samples : 102--
--98th video sampled, samples : 0 / total samples : 102--
--99th video sampled, samples : 3 / total samples : 105--
--100th video sample

In [2]:
import tensorflow as tf
from tensorflow.keras import layers as L
from tensorflow.keras import models as M

import time
import datetime    #second to hour
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, Flatten
#from sklearn.model_selection import train_test_split

#랜덤셔플  
from random import shuffle


from keras import backend as K

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
#K.tensorflow_backend._get_available_gpus()

    
    
    
LABELS = [0,1,2,3,4,5]
size_grid = 640
size_sample = 15 #input size + predict output size(1)  (timesteps)cudnnlstm

#model evaluation

#Let's define size_batch to make batches with the fixed size
size_batch = 20  #should be divided by 2
test_percent = 0.1
size_batch_train = int(size_batch*(1-test_percent))
size_batch_test = int(size_batch*test_percent)

######아 그냥 5 5 해야겠네

t_steps = size_sample
#-1에서 걍 그 값으로 바꿈. x y 다 같아야되는 것 같은듯!
########걍 20개 넣고 y는 1 0 

# define model
#input (batch_size, timesteps, features(data_dim))

model = tf.keras.Sequential()

##retrain
#model = load_model('/home/hyesong/learning_eyes/Activity_Recognition/model50_50_1113.h5') 


#batch_input_.. 포기함.. predict 불가야...
#
#CuDNNLSTM........ 없어 아직 tf 2.0엔 그래서 저거 씀
model.add(tf.compat.v1.keras.layers.CuDNNLSTM(size_sample*2, return_sequences=True, input_shape=(t_steps, size_grid ))) 
model.add(Dropout(0.1))
model.add(tf.compat.v1.keras.layers.CuDNNLSTM(size_sample*2, return_sequences=True))
model.add(Dropout(0.1))
model.add(tf.compat.v1.keras.layers.CuDNNLSTM(size_sample*2))
model.add(L.Dense(len(LABELS), activation='softmax')) #3개 이상부터 softmax? 인듯.. binary까진 sigmoid
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#rmsprop, adam



X = []
X_list = []
y = []
y_list = []


#train start
startTime = time.time()

#define dataset
#batch size = all cases from videos
video_index = 0
n_batch = 0    #same or lower than size_batch 

total_labels = len(total_data)

#################



#이거 원래 class 3개부터 쓰는거라
v_labels = to_categorical(LABELS, num_classes = len(LABELS)) 

print(v_labels)



#데이터셋 골고루 섞은 버전. 랜덤셔플
shuffledData = []
for label_i in LABELS:
    for sample in total_data[label_i][1]:
        shuffledData.append([label_i, sample])  #샘플이 타임스텝개 되어있는 그거
        
        
shuffle(shuffledData)

  
X_train = []
y_train = []
X_test = []
y_test = []    

for i in range(len(shuffledData)):

    n_batch += 1

    #for k in range(t_steps):  #label에서의 size_sample개의 변화가 들어있는 샘플셋
    X_list.append(shuffledData[i][1])
       
            
    ######X, y dataset 형태 꼭 제대로 확인
    y_list.append(v_labels[shuffledData[i][0]])
    #print(shuffledData[i][1])
            #outputs.append(label_index)
  
       # X_list.append(inputs)  
        #xl = int((len(X_list)))
        
    if (n_batch%size_batch == 0) :
          
        for i in range(size_batch_train):
            X_train.append(X_list[i])
            y_train.append(y_list[i])
                
        for i in range(size_batch_train, size_batch):            
            X_test.append(X_list[i])
            y_test.append(y_list[i])
            
        X_list = []
        y_list = [] 

            
        
#reshape
X_train = array(X_train)
y_train = array(y_train) 
X_test = array(X_test)
y_test = array(y_test)
    
print(X_test.shape)
print(y_test.shape)
            
#X_train = X_train.reshape((size_batch_train, t_steps, size_grid)) 
#X_test = X_test.reshape((size_batch_test, t_steps, size_grid))
# fit model ... steps_per_epoch=size_sample

history = model.fit(X_train, y_train, batch_size = 1, epochs=50, validation_data=(X_test, y_test),verbose=1)            
            
#이 버전은 내가 ㄹ알아서돌리는거같은데 코드로
        
    
        
 
            
  
endTime = time.time() - startTime
hourtime = str(datetime.timedelta(seconds = endTime))
print("====== " + hourtime + " ======")
    
    
#and save(keras, with 'save', architecture and model are saved simultaneously)
name = str('model50_50_fin.h5')
model.save(name)

    
print("------------All train finished---------------")

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3651390039644828688
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5096459330447734239
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10506240000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10877445658306804820
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5067517627190350482
physical_device_desc: "device: XLA_GPU device"
]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 15, 30)            80640     
_________________________________________________________________
dropout 

Epoch 43/50
1296/1296 [==============================] - 5s 4ms/sample - loss: 0.1626 - accuracy: 0.9498 - val_loss: 3.9538 - val_accuracy: 0.5069
Epoch 44/50
1296/1296 [==============================] - 5s 4ms/sample - loss: 0.1604 - accuracy: 0.9452 - val_loss: 4.0460 - val_accuracy: 0.5278
Epoch 45/50
1296/1296 [==============================] - 5s 4ms/sample - loss: 0.1587 - accuracy: 0.9444 - val_loss: 3.8110 - val_accuracy: 0.5625
Epoch 46/50
1296/1296 [==============================] - 6s 5ms/sample - loss: 0.1646 - accuracy: 0.9514 - val_loss: 4.5115 - val_accuracy: 0.5208
Epoch 47/50
1296/1296 [==============================] - 7s 5ms/sample - loss: 0.1654 - accuracy: 0.9491 - val_loss: 4.3885 - val_accuracy: 0.5069
Epoch 48/50
1296/1296 [==============================] - 5s 4ms/sample - loss: 0.1623 - accuracy: 0.9491 - val_loss: 4.5394 - val_accuracy: 0.5069
Epoch 49/50
1296/1296 [==============================] - 6s 5ms/sample - loss: 0.1681 - accuracy: 0.9437 - val_loss: 3